In [1]:
from datetime import datetime
import json
import requests
import numpy as np

import checklist
from checklist.expect import Expect
from checklist.test_types import MFT, INV, DIR

In [2]:
INTENTS = [
    'book-appointment',
    'book-hotel',
    'book-restaurant',
    'change-appointment',
    'change-car-rental',
    'change-driver',
    'change-hotel',
    'change-reminder',
    'change-restaurant',
    'contact-provider',
    'context',
    'find-Property',
    'find-Recipe',
    'find-Trip',
    'find-activity',
    'find-around-me',
    'find-car-rental',
    'find-driver',
    'find-flight',
    'find-hotel',
    'find-information',
    'find-itinerary',
    'find-restaurant',
    'find-train',
    'inform',
    'irrelevant',
    'other',
    'provide-agenda',
    'provide-news',
    'provide-showtimes',
    'provide-translation',
    'provide-tv-guide',
    'provide-weather',
    'purchase',
    'set-reminder',
    'smalltalk',
    'unsupported',
]

In [3]:
#def get_classif(classification):
#    return INTENTS.index(max(classification, key=classification.get))

In [4]:
def get_classif(classification):
    value = INTENTS.index(max(classification, key=classification.get))
    if value not in [15, 18, 19, 22, 23, 25, 29, 33] :
        value = 25
    return value

In [5]:
def get_softmax(classification):
    return [c/100 for c in classification.values()]

In [6]:
def get_classification(sentences):
    URL = "http://teaching-pfe.francecentral.azurecontainer.io/api/classification"  
    LANG = "fr-FR"
    classifs = [requests.get(f"{URL}?iso_language={LANG}&sentence={sentence}").json().get("classification") for sentence in sentences]
    return (np.array([get_classif(c) for c in classifs]), np.array([get_softmax(c) for c in classifs]))

In [7]:
with open("dataset_aug.json", "r") as dataset_file:
   data = json.load(dataset_file)

print(data[0])

{'intent': 'irrelevant', 'sentence': 'Est ce que tu peux me dire en détails ce que tu sais faire ?', 'keyboard_aug': 'Est ce que tu peux me sire en détaums ce que tu sais faire?', 'wordswap_aug': 'Est ce que tu peux me dire détails en que ce sais tu? faire'}


# Create and run tests

## MFT

### Original dataset

In [50]:
data_mft_original = { "data": [elem.get("sentence") for elem in data],
                      "labels": [INTENTS.index(elem.get("intent")) for elem in data]}

label_0 = data_mft_original.get('labels')[0]
print(f" Sample - data : {data_mft_original.get('data')[0]}; label : {label_0}/{INTENTS[label_0]}")

 Sample - data : Est ce que tu peux me dire en détails ce que tu sais faire ?; label : 25/irrelevant


In [51]:
test_MFT_original = MFT(**data_mft_original,name="original dataset", capability="None", description=" ")

In [52]:
start = datetime.now()
print(f"Started at {start}")
test_MFT_original.run(get_classification, overwrite=True)
end = datetime.now()
print(f"Execution time : {end - start}") # estimation : 54min

Started at 2020-12-09 13:32:16.972415
Predicting 5395 examples
Execution time : 0:59:55.001080


In [53]:
test_MFT_original.summary()

Test cases:      5395
Fails (rate):    19 (0.4%)

Example fails:
25 (0.0) Fais moi une précommande de la prochaine xbox
----
25 (0.4) j'ai besoin que tu me cherche un b&b pour le dernier weekend du mois à saint-petersbourg
----
19 (0.8) chambre pour cinq aix en provence (sans budget)
----


### Typographie faults


In [54]:
data_mft_typo = { "data": [elem.get("keyboard_aug") for elem in data],
                  "labels": [INTENTS.index(elem.get("intent")) for elem in data]}

label_0 = data_mft_typo.get('labels')[0]
print(f" Sample - data : {data_mft_typo.get('data')[0]}; label : {label_0}/{INTENTS[label_0]}")

 Sample - data : Est ce que tu peux me sire en détaums ce que tu sais faire?; label : 25/irrelevant


In [55]:
test_MFT_typo = MFT(**data_mft_typo, name="typo", capability="None", description="MFT data aug with typo")

In [58]:
start = datetime.now()
print(f"Started at {start}")
test_MFT_typo.run(get_classification, overwrite=True)
end = datetime.now()
print(f"Execution time : {end - start}") # estimation : 54min

Started at 2020-12-09 15:36:17.022950
Predicting 5395 examples
Execution time : 0:51:13.389092


In [59]:
test_MFT_typo.summary()

Test cases:      5395
Fails (rate):    430 (8.0%)

Example fails:
25 (0.1) y a un bon itaOieb Fers moi?
----
25 (0.0) Trouvd un hypnologue avec des consultations abordqnles et efficaces
----
25 (0.8) chercteG restaurant
----


### Switching words' positions

In [61]:
data_mft_switch = { "data": [elem.get("wordswap_aug") for elem in data],
                  "labels": [INTENTS.index(elem.get("intent")) for elem in data]}
            
label_0 = data_mft_switch.get('labels')[0]
print(f" Sample - data : {data_mft_switch.get('data')[0]}; label : {label_0}/{INTENTS[label_0]}")

 Sample - data : Est ce que tu peux me dire détails en que ce sais tu? faire; label : 25/irrelevant


In [62]:
test_MFT_switch = MFT(**data_mft_switch, name="switch", capability="None", description=" data aug with switch")

In [63]:
start = datetime.now()
print(f"Started at {start}")
test_MFT_switch.run(get_classification, overwrite=True)
end = datetime.now()
print(f"Execution time : {end - start}")  # estimation : 54min

Started at 2020-12-09 16:58:03.504773
Predicting 5395 examples
Execution time : 0:54:44.174924


In [65]:
test_MFT_switch.summary()

Test cases:      5395
Fails (rate):    95 (1.8%)

Example fails:
25 (0.0) Cherche terrain de badminton jeu libre / 5 € p notre vers d dame ' oé
----
15 (0.9) Dermatologue un au plus vite
----
25 (0.0) 2 lyon, psychiatre
----


## INV

### Typographie faults

In [67]:
t_typo = { "data":[[item.get("sentence"), item.get("keyboard_aug")] for item in data]}

print(f"samples - {t_typo.get('data')[:2]}")

samples - [['Est ce que tu peux me dire en détails ce que tu sais faire ?', 'Est ce que tu peux me sire en détaums ce que tu sais faire?'], ["J'ai faim !!! Propose moi quelque chose, mais pas de fast food car j'ai déjà mangé à Quick hier !", "J ' ai faim! !! Propose moi auelWue chose, ùais pas de fast food car j ' ai déjà mangé à Quick hier!"]]


In [68]:
test_INV_typo = INV(**t_typo, name="typo augmentation", description="test with data augmentation with typo")

In [69]:
start = datetime.now()
print(f"Started at {start}")
test_INV_typo.run(get_classification, overwrite=True)
end = datetime.now()
print(f"Execution time : {end - start}")  # estimation : 2h

Started at 2020-12-09 17:59:16.403402
Predicting 10790 examples
Execution time : 1:53:41.151320


In [70]:
test_INV_typo.summary()

Test cases:      5395
Fails (rate):    280 (5.2%)

Example fails:
22 (1.0) Peux-tu me suggérer un restaurant ?
15 (0.5) 0eux - tu me suggérer un restZur&nt?

----
22 (1.0) bonjour, tu me trouves un resto d'asie avec menu à 15 euros ? Merci !
33 (0.0) bonjour, tu me trouves un resgo d ' asie aves menu à 15 euros? Merci!

----
15 (1.0) Trouve-moi l'Auchan le plus proche
25 (0.0) TrKuve - moi l ' Auchan le plus prochS

----


### Switching words' positions

In [8]:
t_switch = { "data":[[item.get("sentence"), item.get("wordswap_aug")] for item in data]}

print(f"samples - {t_switch.get('data')[:2]}")

samples - [['Est ce que tu peux me dire en détails ce que tu sais faire ?', 'Est ce que tu peux me dire détails en que ce sais tu? faire'], ["J'ai faim !!! Propose moi quelque chose, mais pas de fast food car j'ai déjà mangé à Quick hier !", "' j ai faim! !! Propose quelque moi chose, mais pas fast de car food j ai ' déjà à mangé Quick hier!"]]


In [9]:
test_INV_switch = INV(**t_switch, name="switch augmentation", description="test with data augmentation with swap words")

In [11]:
start = datetime.now()
print(f"Started at {start}")
test_INV_switch.run(get_classification, overwrite=True)
end = datetime.now()
print(f"Execution time : {end - start}")  # estimation : 2h

Started at 2020-12-09 22:58:49.941000
Predicting 10790 examples
Execution time : 1:41:25.283714


In [12]:
test_INV_switch.summary()

Test cases:      5395
Fails (rate):    50 (0.9%)

Example fails:
22 (0.9) Est-ce que le restaurant le fifties au Havre est ouvert ce soir et quelle est la carte des menus ?
25 (0.0) Est - que ce le restaurant au le fifties est Havre ouvert ce soir et est quelle carte la des menus?

----
19 (1.0) Il me faut 1 chambre avec lit-double aux alentours de Manosque le 15 août pour le mariage de ma sœur.
25 (0.0) Il me faut 1 chambre lit avec - double alentours de aux le Manosque 15 août le pour mariage de ma. sœur

----
19 (1.0) Hôtel à 100 euros par nuit
25 (0.0) À hôtel 100 euros par nuit

----


# Visualisation

## Original dataset

In [44]:
test_MFT_original.visual_summary()

TestSummarizer(stats={'npassed': 10, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': 'original dataset', 'd…

## Typo

In [60]:
test_MFT_typo.visual_summary()

TestSummarizer(stats={'npassed': 4965, 'nfailed': 430, 'nfiltered': 0}, summarizer={'name': 'typo', 'descripti…

In [71]:
test_INV_typo.visual_summary()

TestSummarizer(stats={'npassed': 5115, 'nfailed': 280, 'nfiltered': 0}, summarizer={'name': 'typo augmentation…

## Switch



In [66]:
test_MFT_switch.visual_summary()

TestSummarizer(stats={'npassed': 5300, 'nfailed': 95, 'nfiltered': 0}, summarizer={'name': 'switch', 'descript…

In [13]:
test_INV_switch.visual_summary()

TestSummarizer(stats={'npassed': 5345, 'nfailed': 50, 'nfiltered': 0}, summarizer={'name': 'switch augmentatio…

TestSummarizer(stats={'npassed': 6, 'nfailed': 4, 'nfiltered': 0}, summarizer={'name': 'typo augmentation', 'd…